In [1]:
import pandas as pd

**IMDB Movie Review Datset** \
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. the dataset consists of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. This data is organized into pos and neg folders with one text file per example.\ https://ai.stanford.edu/~amaas/data/sentiment/

In [2]:
# Creating a function to read the data into train and test sets

from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('dataset/aclImdb/train')
test_texts, test_labels = read_imdb_split('dataset/aclImdb/test')

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_139126/51250322.py:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  labels.append(0 if label_dir is "neg" else 1)


Creating a validation set from the training set for evaluation and model tuning without feeding the model the test set


In [3]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)